<a href="https://colab.research.google.com/github/gandhimonil9823/CS_GO_Analysis/blob/main/Algorithm_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [136]:
import pandas as pd
import os
import matplotlib.pyplot as plt
import seaborn as sns
plt.style.use('fivethirtyeight')
plt_kwargs = {'figsize': (10, 4)}
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report, f1_score, confusion_matrix
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn import metrics
from sklearn.model_selection import cross_val_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve
from sklearn import model_selection
from sklearn import svm
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV, RandomizedSearchCV
from sklearn.neighbors import KNeighborsClassifier
from xgboost import XGBClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn import preprocessing
from sklearn.tree import DecisionTreeClassifier


In [14]:
from google.colab import drive
drive.mount('/content/drive')
#drive.mount(r'drive/folders/0AAxMXLC9WPNxUk9PVA')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [16]:
ls

aggregated_data/  de_inferno.png    de_season.png     mm_grenades_demos.csv
cs_cruise.png     de_mikla.png      de_train.png      out-de_cache.png
damage_data.csv   de_mirage.png     de_tulip.png      out-de_cbble.png
de_cache.png      de_new_nuke.png   grenade_data.csv  out-de_dust2.png
de_cbble.png      de_nuke.png       kill_data.csv     out-de_inferno.png
de_coast.png      de_overpass.png   Log_ROC.png       out-de_mirage.png
de_dust2.png      de_royal.png      map_data.csv      out-de_overpass.png
de_empire.png     de_santorini.png  meta_data.csv     out-de_train.png


In [17]:
cd 'drive/Shared drives/CS_GO_Data/processed_data_v2'

[Errno 2] No such file or directory: 'drive/Shared drives/CS_GO_Data/processed_data_v2'
/content/drive/Shared drives/CS_GO_Data/processed_data_v2


In [18]:
ls

aggregated_data/  de_inferno.png    de_season.png     mm_grenades_demos.csv
cs_cruise.png     de_mikla.png      de_train.png      out-de_cache.png
damage_data.csv   de_mirage.png     de_tulip.png      out-de_cbble.png
de_cache.png      de_new_nuke.png   grenade_data.csv  out-de_dust2.png
de_cbble.png      de_nuke.png       kill_data.csv     out-de_inferno.png
de_coast.png      de_overpass.png   Log_ROC.png       out-de_mirage.png
de_dust2.png      de_royal.png      map_data.csv      out-de_overpass.png
de_empire.png     de_santorini.png  meta_data.csv     out-de_train.png


In [109]:
df_winner_matches_agg = pd.read_csv("aggregated_data/modelinput/winner_matches_agg.csv")

df_winner_side_rounds_agg = pd.read_csv("aggregated_data/modelinput/winner_side_rounds_agg.csv")

df_winner_team_rounds_agg = pd.read_csv("aggregated_data/modelinput/winner_team_rounds_agg.csv")




# Data Preprocess for Data Mining Algorithms


In [110]:
df_esea_meta_demos =  pd.read_csv("meta_data.csv")

df_esea_meta_demos.head()

,file,map,round,start_seconds,end_seconds,winner_team,winner_side,round_type,ct_eq_val,t_eq_val
0,esea_match_13770997.dem,de_overpass,1,94.30782,160.9591,Team 2,Terrorist,PISTOL_ROUND,4300,4250
1,esea_match_13770997.dem,de_overpass,2,160.95910,279.3998,Team 2,Terrorist,ECO,6300,19400
2,esea_match_13770997.dem,de_overpass,3,279.39980,341.0084,Team 2,Terrorist,SEMI_ECO,7650,19250
3,esea_match_13770997.dem,de_overpass,4,341.00840,435.4259,Team 2,Terrorist,NORMAL,24900,23400
4,esea_match_13770997.dem,de_overpass,5,435.42590,484.2398,Team 1,CounterTerrorist,ECO,5400,20550


In [111]:
def terrorist_counterT_win(series_shifted_winner_side):
  terrorist_win = series_shifted_winner_side.apply(lambda x: 1 if x=='Terrorist' else 0)
  terrorist_win = terrorist_win.iloc[:-1]
  terrorist_win = terrorist_win.reset_index(drop=True)

  counterterrorist_win = series_shifted_winner_side.apply(lambda x: 1 if x=='CounterTerrorist' else 0)
  counterterrorist_win = counterterrorist_win.iloc[:-1]
  counterterrorist_win = counterterrorist_win.reset_index(drop=True)

  return terrorist_win, counterterrorist_win

def preprocess_meta_demos_algo(df_esea_meta_demos):

  df_original_preprocessed = df_esea_meta_demos[['round','ct_eq_val','t_eq_val']]

  unique_maps = df_esea_meta_demos["map"].unique()

  map_dataframe = pd.DataFrame()

  for map_cs_go in unique_maps:
    map_dataframe[map_cs_go] = df_esea_meta_demos["map"].apply(lambda x: 1 if x == map_cs_go else 0)

  series_one_element = pd.Series([0])

  binary_values_first_round = df_esea_meta_demos["round"].apply(lambda x : 0 if x == 1 else 1)

  binary_values_first_round = binary_values_first_round.rename("not_First_Round").reset_index(drop=True)

  series_shifted_winner_side = series_one_element.append(df_esea_meta_demos["winner_side"])

  terrorist_win, counterterrorist_win = terrorist_counterT_win(series_shifted_winner_side)

  terrorist_win = (terrorist_win * binary_values_first_round).rename('terrorist_win')

  counterterrorist_win = (counterterrorist_win * binary_values_first_round).rename('counterterrorist_win')

  previous_win_df = pd.concat([terrorist_win, counterterrorist_win])

  data_to_predict_T_CT = df_esea_meta_demos['winner_side'].apply(lambda x: 1 if x=='Terrorist' else 0)

  column_names = [df_original_preprocessed, map_dataframe, terrorist_win, counterterrorist_win, data_to_predict_T_CT]

  return pd.concat(column_names, axis = 'columns', join = 'inner')

In [129]:
df_meta_data = preprocess_meta_demos_algo(df_esea_meta_demos)

agg_df_rounds = pd.concat([df_winner_side_rounds_agg, df_meta_data[['terrorist_win', 'counterterrorist_win',]]], axis=1)

agg_df_rounds



,round,start_seconds,end_seconds,round_duration,ct_eq_val,t_eq_val,is_bomb_planted,bomb_site,past_halftime,de_overpass,de_cache,de_mirage,de_inferno,de_train,de_dust2,de_cbble,de_nuke,PISTOL_ROUND,ECO,SEMI_ECO,NORMAL,FORCE_BUY,t_lethal_nades_eff,ct_lethal_nades_eff,t_util_nades_eff,ct_util_nades_eff,winner_side_class,terrorist_win,counterterrorist_win
0,1,94.30782,160.9591,66.65128,4300,4250,1,-1,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,-1,0,0
1,2,160.95910,279.3998,118.44070,6300,19400,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,3,2,1,2,-1,1,0
2,3,279.39980,341.0084,61.60860,7650,19250,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,1,0,1,0,-1,1,0
3,4,341.00840,435.4259,94.41750,24900,23400,1,1,0,1,0,0,0,0,0,0,0,0,0,0,1,0,2,10,8,1,-1,1,0
4,5,435.42590,484.2398,48.81390,5400,20550,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,1,0,2,1,1,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
340328,18,1647.20600,1694.6550,47.44900,4500,21950,0,0,1,0,0,1,0,0,0,0,0,0,1,0,0,0,1,0,3,0,-1,1,0
340329,19,1694.65500,1753.4380,58.78300,16400,26100,0,0,1,0,0,1,0,0,0,0,0,0,0,0,1,0,3,2,6,3,-1,1,0
340330,20,1753.43800,1803.6580,50.22000,15100,26000,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,1,0,1,4,4,-1,1,0
340331,21,1803.65800,1856.0850,52.42700,17550,25500,0,0,1,0,0,1,0,0,0,0,0,0,0,0,1,0,3,3,2,3,-1,1,0


In [130]:
def plot_hist(df):
  arr = ['LogReg', 'RF', 'KNN', 'SVM', "XGB"]
  for i, col in enumerate(df.columns):
    plt.figure(i)
    sns.distplot(df[col])
  plt.xlabel(["fit_time_average","score_time_average","test_accuracy_average","test_precision_weighted_average","test_recall_weighted_average","test_f1_weighted_average", "test_roc_auc_average"])


# Models for WinnerSide based on Rounds

In [137]:
X = agg_df_rounds.drop('winner_side_class', axis = 1)[:1000]

y = agg_df_rounds['winner_side_class'][:1000]

# using train_test_split for splitting the data into four different sets with 20% for the test data set
X_train, X_test, Y_train, Y_test = train_test_split(X, y, test_size=0.20, random_state=1)

# scale the data
standard_scalar = StandardScaler()

X_train = standard_scalar.fit_transform(X_train)

X_test = standard_scalar.fit_transform(X_test)

models = [
            ('LogReg', LogisticRegression()), 
            ('RF', RandomForestClassifier(max_depth=2, random_state=0)),
            ('KNN', KNeighborsClassifier(n_neighbors=8)),
            ('SVM', svm.SVC(kernel='linear', C=0.2)),
            ("XGB", XGBClassifier(learning_rate=0.1,max_depth=4,n_estimators=100)),
            ("ADA", AdaBoostClassifier(base_estimator=DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None, criterion='gini',max_depth=3, 
                                                                             max_features=None, max_leaf_nodes=None,
                                                                             min_impurity_decrease=0.0, 
                                                                             min_impurity_split=None,
                                                                             min_samples_leaf=1, 
                                                                             min_samples_split=2,
                                                                             min_weight_fraction_leaf=0.0, 
                                                                             presort='deprecated', 
                                                                             random_state=None, 
                                                                             splitter='best'), 
                                                                              learning_rate=0.5, n_estimators=10))]

In [138]:
def run_models(X_train, X_test, Y_train, Y_test, models):
  scoring = ['accuracy', 'precision_weighted', 'recall_weighted', 'f1_weighted', 'roc_auc']
  dfs = []  
  results = []
  names = []
  for name, model in models:
    kfold = model_selection.KFold(n_splits=6, shuffle=True, random_state=90210)
    cv_results = model_selection.cross_validate(model, X_train, Y_train, cv=kfold, scoring=scoring)
    clf = model.fit(X_train, Y_train)
    y_pred = clf.predict(X_test)
    print(name)
    print(classification_report(Y_test, y_pred))
    results.append(cv_results)
    this_df = pd.DataFrame(cv_results)
    this_df['model'] = name
    dfs.append(this_df)
  final = pd.concat(dfs, ignore_index=True)
  return final

In [139]:
df_kfold_results = run_models(X_train, X_test, Y_train, Y_test, models)

df_kfold_results_average_values = df_kfold_results.groupby('model').mean()

df_kfold_results_average_values.columns = [str(col) + '_average' for col in df_kfold_results_average_values.columns]

df_kfold_results_average_values

LogReg
              precision    recall  f1-score   support

          -1       0.76      0.73      0.75       107
           1       0.70      0.74      0.72        93

    accuracy                           0.73       200
   macro avg       0.73      0.74      0.73       200
weighted avg       0.74      0.73      0.74       200

RF
              precision    recall  f1-score   support

          -1       0.76      0.81      0.79       107
           1       0.77      0.71      0.74        93

    accuracy                           0.77       200
   macro avg       0.77      0.76      0.76       200
weighted avg       0.77      0.77      0.76       200

KNN
              precision    recall  f1-score   support

          -1       0.71      0.70      0.71       107
           1       0.66      0.68      0.67        93

    accuracy                           0.69       200
   macro avg       0.69      0.69      0.69       200
weighted avg       0.69      0.69      0.69       200

SVM
 

,fit_time_average,score_time_average,test_accuracy_average,test_precision_weighted_average,test_recall_weighted_average,test_f1_weighted_average,test_roc_auc_average
model,,,,,,,
ADA,0.038095,0.008042,0.758772,0.765174,0.758772,0.759117,0.818871
KNN,0.002149,0.019303,0.699996,0.711327,0.699996,0.699893,0.775734
LogReg,0.014867,0.005940,0.763738,0.771788,0.763738,0.764104,0.836411
RF,0.166328,0.023977,0.764972,0.778541,0.764972,0.764703,0.843969
SVM,0.019430,0.007186,0.772463,0.784709,0.772463,0.772065,0.827550
XGB,0.115730,0.006414,0.758744,0.765022,0.758744,0.759183,0.845862


# Models for WinnerSide based on Matches

In [121]:
def run_models_multiclass(X_train, X_test, Y_train, Y_test, models):
  scoring = ['accuracy', 'precision_weighted', 'recall_weighted', 'f1_weighted']
  dfs = []  
  results = []
  names = []
  for name, model in models:
    kfold = model_selection.KFold(n_splits=6, shuffle=True, random_state=90210)
    cv_results = model_selection.cross_validate(model, X_train, Y_train, cv=kfold, scoring=scoring)
    clf = model.fit(X_train, Y_train)
    y_pred = clf.predict(X_test)
    print(name)
    print(classification_report(Y_test, y_pred))
    results.append(cv_results)
    this_df = pd.DataFrame(cv_results)
    this_df['model'] = name
    dfs.append(this_df)
  final = pd.concat(dfs, ignore_index=True)
  return final

In [140]:
df_winner_matches_agg = df_winner_matches_agg.dropna()

X = df_winner_matches_agg.drop('winner_class', axis = 1)[:10000]


y = df_winner_matches_agg['winner_class'][:10000]

# y = preprocessing.label_binarize(df_winner_matches_agg['winner_class'][:10000], classes=[0, 1, -1])


# using train_test_split for splitting the data into four different sets with 20% for the test data set
X_train, X_test, Y_train, Y_test = train_test_split(X, y, test_size=0.20, random_state=1)

# scale the data
standard_scalar = StandardScaler()

X_train = standard_scalar.fit_transform(X_train)

X_test = standard_scalar.fit_transform(X_test)

models = [
            ('LogReg', LogisticRegression(multi_class="ovr")),
            ('RF', RandomForestClassifier(max_depth=2, random_state=0)),
            ('KNN', KNeighborsClassifier(n_neighbors=8)),
            ('SVM', svm.SVC(kernel='linear', C=1)),
            ("XGB", XGBClassifier(learning_rate=0.1,max_depth=4,n_estimators=100, objective = 'multi:softmax')),
            ("ADA", AdaBoostClassifier(base_estimator=DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None, criterion='gini',max_depth=3, 
                                                                             max_features=None, max_leaf_nodes=None,
                                                                             min_impurity_decrease=0.0, 
                                                                             min_impurity_split=None,
                                                                             min_samples_leaf=1, 
                                                                             min_samples_split=2,
                                                                             min_weight_fraction_leaf=0.0, 
                                                                             presort='deprecated', 
                                                                             random_state=None, 
                                                                             splitter='best'), 
                                                                              learning_rate=0.5, n_estimators=10))]

In [141]:
df_kfold_results = run_models_multiclass(X_train, X_test, Y_train, Y_test, models)

df_kfold_results_average_values = df_kfold_results.groupby('model').mean()

df_kfold_results_average_values.columns = [str(col) + '_average' for col in df_kfold_results_average_values.columns]

df_kfold_results_average_values

LogReg
              precision    recall  f1-score   support

          -1       0.46      0.28      0.35       951
           0       0.00      0.00      0.00        67
           1       0.50      0.73      0.59       982

    accuracy                           0.49      2000
   macro avg       0.32      0.33      0.31      2000
weighted avg       0.47      0.49      0.46      2000



/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Pr

RF
              precision    recall  f1-score   support

          -1       0.45      0.10      0.16       951
           0       0.00      0.00      0.00        67
           1       0.50      0.92      0.65       982

    accuracy                           0.50      2000
   macro avg       0.32      0.34      0.27      2000
weighted avg       0.46      0.50      0.40      2000



/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


KNN
              precision    recall  f1-score   support

          -1       0.48      0.54      0.51       951
           0       0.15      0.10      0.12        67
           1       0.51      0.46      0.48       982

    accuracy                           0.48      2000
   macro avg       0.38      0.37      0.37      2000
weighted avg       0.48      0.48      0.48      2000



/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Pr

SVM
              precision    recall  f1-score   support

          -1       0.43      0.06      0.10       951
           0       0.00      0.00      0.00        67
           1       0.49      0.93      0.64       982

    accuracy                           0.49      2000
   macro avg       0.31      0.33      0.25      2000
weighted avg       0.45      0.49      0.36      2000



/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


XGB
              precision    recall  f1-score   support

          -1       0.52      0.34      0.41       951
           0       0.31      0.22      0.26        67
           1       0.54      0.73      0.62       982

    accuracy                           0.53      2000
   macro avg       0.46      0.43      0.43      2000
weighted avg       0.52      0.53      0.51      2000

ADA
              precision    recall  f1-score   support

          -1       0.46      0.28      0.35       951
           0       0.29      0.06      0.10        67
           1       0.52      0.74      0.61       982

    accuracy                           0.50      2000
   macro avg       0.42      0.36      0.35      2000
weighted avg       0.48      0.50      0.47      2000



,fit_time_average,score_time_average,test_accuracy_average,test_precision_weighted_average,test_recall_weighted_average,test_f1_weighted_average
model,,,,,,
ADA,0.130152,0.008468,0.506378,0.488680,0.506378,0.482747
KNN,0.050944,0.132593,0.499377,0.503329,0.499377,0.496642
LogReg,0.048891,0.005482,0.502747,0.473179,0.502747,0.470645
RF,0.334171,0.021313,0.512751,0.476917,0.512751,0.399992
SVM,1.374340,0.122321,0.506375,0.475227,0.506375,0.376956
XGB,1.218566,0.014823,0.516875,0.503260,0.516875,0.498494


# Models for WinnerSide based on Teams

In [142]:
X = df_winner_team_rounds_agg.drop('winner_class', axis = 1)[:10000]

y = df_winner_team_rounds_agg['winner_class'][:10000]

# using train_test_split for splitting the data into four different sets with 20% for the test data set
X_train, X_test, Y_train, Y_test = train_test_split(X, y, test_size=0.20, random_state=1)

# scale the data
standard_scalar = StandardScaler()

X_train = standard_scalar.fit_transform(X_train)

X_test = standard_scalar.fit_transform(X_test)

models = [
            ('LogReg', LogisticRegression(multi_class="ovr")),
            ('RF', RandomForestClassifier(max_depth=2, random_state=0)),
            ('KNN', KNeighborsClassifier(n_neighbors=8)),
            ('SVM', svm.SVC(kernel='linear', C=0.2)),
            ("XGB", XGBClassifier(learning_rate=0.1,max_depth=4,n_estimators=100)),
            ("ADA", AdaBoostClassifier(base_estimator=DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None, criterion='gini',max_depth=3, 
                                                                             max_features=None, max_leaf_nodes=None,
                                                                             min_impurity_decrease=0.0, 
                                                                             min_impurity_split=None,
                                                                             min_samples_leaf=1, 
                                                                             min_samples_split=2,
                                                                             min_weight_fraction_leaf=0.0, 
                                                                             presort='deprecated', 
                                                                             random_state=None, 
                                                                             splitter='best'), 
                                                                              learning_rate=0.5, n_estimators=10))]

In [144]:
df_kfold_results = run_models(X_train, X_test, Y_train, Y_test, models)

df_kfold_results_average_values = df_kfold_results.groupby('model').mean()

df_kfold_results_average_values.columns = [str(col) + '_average' for col in df_kfold_results_average_values.columns]

df_kfold_results_average_values

LogReg
              precision    recall  f1-score   support

          -1       0.54      0.57      0.56       993
           1       0.55      0.52      0.54      1007

    accuracy                           0.55      2000
   macro avg       0.55      0.55      0.55      2000
weighted avg       0.55      0.55      0.55      2000

RF
              precision    recall  f1-score   support

          -1       0.71      0.79      0.75       993
           1       0.77      0.69      0.73      1007

    accuracy                           0.74      2000
   macro avg       0.74      0.74      0.74      2000
weighted avg       0.74      0.74      0.74      2000

KNN
              precision    recall  f1-score   support

          -1       0.71      0.77      0.74       993
           1       0.75      0.69      0.72      1007

    accuracy                           0.73      2000
   macro avg       0.73      0.73      0.73      2000
weighted avg       0.73      0.73      0.73      2000

SVM
 

,fit_time_average,score_time_average,test_accuracy_average,test_precision_weighted_average,test_recall_weighted_average,test_f1_weighted_average,test_roc_auc_average
model,,,,,,,
ADA,0.255521,0.014450,0.760749,0.760894,0.760749,0.760753,0.825935
KNN,0.056413,0.783837,0.744498,0.745894,0.744498,0.743985,0.813239
LogReg,0.037477,0.008255,0.542500,0.542644,0.542500,0.541938,0.558252
RF,0.393312,0.037523,0.741250,0.742974,0.741250,0.740552,0.803030
SVM,2.056763,0.464009,0.542250,0.542847,0.542250,0.542024,0.558074
XGB,0.819326,0.018127,0.773498,0.773646,0.773498,0.773508,0.845042
